In [ ]:
# !pip install pandas==1.3.4
# !pip install plotly==5.3.1
!pip install kaleido==0.2.1

     |████████████████████████████████| 79.9 MB 1.2 MB/s 


In [7]:
from google.colab import drive
drive.mount('/content/drive')

# Import LSTNet (https://github.com/flaviagiammarino/lstnet-tensorflow)
# !ls /content/gdrive/MyDrive/Masterarbeit/Code/lstnet-tensorflow/lstnet_tensorflow/*.py
import sys
sys.path.append('/content/drive/MyDrive/Masterarbeit/Code/lstnet-tensorflow/')
import lstnet_tensorflow

import matplotlib.pyplot as plt 
import numpy as np

# Load data
data1 = np.load('/content/drive/MyDrive/Masterarbeit/Code/two-hearts/sample-data/0001.npy')
data2 = np.load('/content/drive/MyDrive/Masterarbeit/Code/two-hearts/sample-data/0017.npy')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Generate two time series
N = 500
t = np.linspace(0, 1, N)
e = np.random.multivariate_normal(mean=[0, 0], cov=[[1, 0.25], [0.25, 1]], size=N)
a = 40 + 30 * t + 20 * np.cos(2 * np.pi * (10 * t - 0.5)) + e[:, 0]
b = 80 - 20 * t + 10 * np.sin(2 * np.pi * (20 * t - 0.5)) + e[:, 1]

(500,)

In [27]:
import numpy as np
from lstnet_tensorflow.model import LSTNet



a = data1[:10000]
b = data2[:10000]
y = np.hstack([a.reshape(- 1, 1), b.reshape(- 1, 1)])
print(y)
y.shape

[[-0.03374548  0.03431316]
 [-0.02144169  0.04670795]
 [-0.00912922  0.05928629]
 ...
 [-0.02971468 -0.019485  ]
 [-0.02907211 -0.01597748]
 [-0.02986433 -0.0180287 ]]


(10000, 2)

In [29]:
# Fit the model
model = LSTNet(
    y=y,
    forecast_period=200,
    lookback_period=600,
    kernel_size=3,
    filters=4,
    gru_units=4,
    skip_gru_units=3,
    skip=600, # maybe too high
    lags=200,
)

model.fit(
    loss='mse',
    learning_rate=0.01,
    batch_size=128,
    epochs=10,
    validation_split=0.2,
    verbose=1
)

Epoch 1/10
118/118 [==============================] - 214s 2s/step - loss: 0.0668 - val_loss: 0.0200
Epoch 2/10
118/118 [==============================] - 187s 2s/step - loss: 0.0124 - val_loss: 0.0080
Epoch 3/10
118/118 [==============================] - 186s 2s/step - loss: 0.0063 - val_loss: 0.0053
Epoch 4/10
118/118 [==============================] - 187s 2s/step - loss: 0.0051 - val_loss: 0.0050
Epoch 5/10
118/118 [==============================] - 190s 2s/step - loss: 0.0046 - val_loss: 0.0058
Epoch 6/10
118/118 [==============================] - 191s 2s/step - loss: 0.0044 - val_loss: 0.0069
Epoch 7/10
118/118 [==============================] - 188s 2s/step - loss: 0.0047 - val_loss: 0.0047
Epoch 8/10
118/118 [==============================] - 188s 2s/step - loss: 0.0046 - val_loss: 0.0040
Epoch 9/10
118/118 [==============================] - 187s 2s/step - loss: 0.0044 - val_loss: 0.0046
Epoch 10/10
118/118 [==============================] - 189s 2s/step - loss: 0.0044 - val_lo

In [31]:
# Plot the in-sample predictions
predictions = model.predict(index=800)
fig = model.plot_predictions()
#fig.write_image('predictions.png', width=750, height=650)
fig.show()

In [32]:
# Plot the out-of-sample forecasts
forecasts = model.forecast()
fig = model.plot_forecasts()
#fig.write_image('forecasts.png', width=750, height=650)
fig.show()